In [ ]:
import pandas as pd
import numpy as np
import datetime
import statsmodels as sm
import matplotlib.pyplot as plt
import seaborn as sns 
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller

%matplotlib inline

In [ ]:
df_raw = pd.read_csv('zillow_data.csv')
df_raw.head()

In [ ]:
df_Zipcode = pd.read_csv('zip_code_database.csv')

In [ ]:
df_raw.rename(columns={'RegionName': 'Zipcode'}, inplace = True)


In [ ]:
df_Zipcode.rename(columns={'zip':'Zipcode'}, inplace = True)
df_Zipcode = df_Zipcode[['Zipcode', 'latitude', 'longitude', 'irs_estimated_population' ]]
df_Zipcode.head()

In [ ]:
df = df_raw.merge(df_Zipcode, on = 'Zipcode')
df.head()

In [ ]:
df_East = df_raw[df_raw.Zipcode < 50000]
df_Central = df_raw[(df_raw.Zipcode >= 50000) & (df_raw.Zipcode < 80000) ]
df_West = df[(df.Zipcode >= 80000)]
#West coast** 
df_West.head()

In [ ]:
pd.melt(df_raw)

In [ ]:
def get_datetimes(df):
    """
    Takes a dataframe:
    returns only those column names that can be converted into datetime objects 
    as datetime objects.
    NOTE number of returned columns may not match total number of columns in passed dataframe
    """
    
    return pd.to_datetime(df.columns.values[7:], format='%Y-%m')

In [ ]:
get_datetimes(df_raw)

In [ ]:
cali_df = df_raw[(df_raw['State'] == 'CA')]
cali_df.info()
tx_df = df_raw[(df_raw['State'] == 'TX')]
tx_df.info()
fl_df = df_raw[(df_raw['State'] == 'FL')]
fl_df.info()

In [ ]:
cali_df.isnull().sum()

In [ ]:
cali_df.isna().sum().sum()

In [ ]:
cali_df.head()

In [ ]:
#RWM
avg = 0
std = 10 
n_pts = 1000
date_vals = pd.date_range(starts = '04/01/1996')

In [ ]:
# font = {'family' : 'normal',
#         'weight' : 'bold',
#         'size'   : 22}

# matplotlib.rc('font', **font)

# # NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

In [ ]:
# def melt_data(df):
#     """
#     Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.  
#     Returns a long-form datetime dataframe 
#     with the datetime column names as the index and the values as the 'values' column.
    
#     If more than one row is passes in the wide-form dataset, the values column
#     will be the mean of the values from the datetime columns in all of the rows.
#     """
    
#     melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'SizeRank', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
#     melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
#     melted = melted.dropna(subset=['value'])
#     return melted.groupby('time').aggregate({'value':'mean'})